In [ ]:
from IPython.core.display import HTML   
display(HTML("<style>:root { --jp-notebook-max-width: 98% !important; }</style>"))

import warnings
warnings.simplefilter("ignore", UserWarning)

# Pattern Matching on AMD's stock price action

This notebook shows pattern matching on AMD's stock price using L2 similarity search on KDB.AI. 

### Goals 

1. Load Data
1. Create Vector Embeddings
1. Store Embeddings in KDB.AI
1. Search For Similar Sequences To A Target Sequence
1. Delete the KDB.AI Table

---

In [ ]:
## import Modules

import os, time
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import pandas_ta         as ta     ## For VWAP calculation 
import kdbai_client      as kdbai
from   utils             import *  ## Import plotting functions

plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set_theme()

### Connect to KDB.AI Session

To use KDB.AI, you will need two session details - a URL endpoint and an API key. 
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

In [ ]:
####  Create ".env" file from  "env_copy.text"

from os.path import join,dirname
from dotenv  import load_dotenv

# dotenv_path = join(dirname, './.env')
load_dotenv('../.env')

# lazy auth    -- just enter info.. 
# endpoint          ='XXXX.kdb.ai' 
# api_key1          ='YOUR_KDBAI_API_KEY'

KDBAI_ENDPOINT  = os.environ.get("endpoint") 
KDBAI_API_KEY1  = os.environ.get("api_key1")

## Sanity Check 
# print(KDBAI_ENDPOINT)

In [ ]:
# Create session early, check table on your instance
session = kdbai.Session(api_key=KDBAI_API_KEY1, endpoint=KDBAI_ENDPOINT)
print(session.list())

### Define Functions

In [ ]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return display(df.head(3))

def TableCheckDrop(t):
    if t in session.list():
        session.table(t).drop()
        time.sleep(2)
        print ("Table found: Dropped, safe to add:", t) 
    else: print ("No Table found: Safe to add:", t) 

def timeSliceData(t1,t2,dataSlice1=pd.DataFrame,dataSliceOpn=pd.DataFrame()):
    dataP_     = dataSlice1.between_time(t1[0],t1[1]).pivot(index='Date', columns='Time', values='Close'); 
    dataP__    = dataSlice1.between_time(t2[0],t2[1]).pivot(index='Date', columns='Time', values='Close');

    dataP_     = dataP_.div(dataSliceOpn.values, axis=0)
    dataP__    = dataP__.div(dataSliceOpn.values,axis=0)

    dataP_V    = dataSlice1.between_time(t1[0],t1[1]).pivot(index='Date', columns='Time', values='VWAP')
    dataP__V   = dataSlice1.between_time(t2[0],t2[1]).pivot(index='Date', columns='Time', values='VWAP')

    dataP_V    = dataP_V.div(dataSliceOpn.values, axis=0)
    dataP__V   = dataP__V.div(dataSliceOpn.values, axis=0)

    IdxMap     = dataP_.reset_index()["Date"].astype('str').to_dict()

    global findIndex
    def findIndex(value): return next((k for k,v in IdxMap.items() if v == value), None)
    # findIndex('2021-12-01')

    global findDate
    def findDate(value): return IdxMap[value]
    #findDate(240)

    return dataP_, dataP__, dataP_V, dataP__V,  IdxMap      
    

In [ ]:
# check for data
# !ls ../data/
# !ls ../images/

### Dataset Overview

The dataset containing AMD stock prices from January 4th, 2021, to March 1st, 2024, with data points recorded between 
8:00 and 15:00. Additionally, we have preprocessed and cleaned the data for smooth operation in a notebook aimed at 
developers. We chose this time period as we believe that ODTE was influencing the stock and will continue to do so. 
We made a decision that this notebook will not be quantative rigorous. We have look ahead bias and other errors. 
Users can tailor the analysis to their specific needs and preferences.

Added columns to Open, High, Low, Close, Volume data 
- _symIDT   = Symbol, Date, Time 
- HL2       = (High+ Low)/2
- DOW       = Day of Week
- RF        = Thursday or Friday
- WOM       = Week of Month
- OpExRF    = OpEx Thursday or Friday


In [ ]:
# Read directly from the zip file.#
dataPrep              = pd.read_csv("../data/AMD_dataPrep.csv.zip")

### Process The Data

Create two dataframes:

- OHLC DataFrame 
- Filters DataFrame

In [ ]:
cols                  = ["TStamp","Open","High","Low","Close","Volume"]
dataSlice1            = dataPrep[cols].copy()

dataSlice1["TStamp"]  = dataSlice1["TStamp"].astype('datetime64[s]')
dataSlice1['Date']    = pd.to_datetime(dataSlice1["TStamp"].dt.date)
dataSlice1['Time']    = dataSlice1["TStamp"].dt.time
dataSlice1            = dataSlice1.set_index("TStamp") 
dataSlice1['VWAP']    = ta.vwap(dataSlice1['High'], dataSlice1['Low'], dataSlice1['Close'], dataSlice1['Volume'], anchor="D")
dataSlice1            = dataSlice1.between_time('9:30','15:59')
# print(dataSlice1.head(3)

dataSliceOpn          = dataSlice1.between_time('9:30','9:30')["Open"]
# dataSliceOpn.head(3) 

In [ ]:
cols                  = ['Symbol', 'TStamp', 'DOW', 'RF', 'WOM', 'OpExRF']
filtersData           = dataPrep[cols].copy()

filtersData["TStamp"] = filtersData["TStamp"].astype('datetime64[s]')
filtersData['Date']   = pd.to_datetime(filtersData["TStamp"].dt.date)
filtersData           = filtersData.set_index("TStamp").between_time('9:30','9:30').set_index("Date")
# filtersData.head(3)

### Select The Time Windows And Create Embeddings

In [ ]:
##Select Time Tuples from 
##  Use 0,1 for mapped ('9:30','9:59:59'),                    and projected ('9:30','12:29:59')
##  Use 1,2 for mapped ('9:30','12:29:59')                    and projected ('9:30','15:59:59')
##  Use 3,1 for mapped ['9:45','10:29:59',"<<--custom time"]  and projected ('9:30','12:29:59')

mapped                = 0   ## mapped time period  
projected             = 1   ## projected time period 

#### adjust           ['9:45','10:29:59',"<<--custom time"] for custom time periods
tPeriod               = [('9:30','9:59:59'),('9:30','12:29:59'),('9:30','15:59:59'),['9:45','10:29:59',"<<--custom time"] ][mapped]
tPeriod2              = [('9:30','9:59:59'),('9:30','12:29:59'),('9:30','15:59:59'),['9:59','15:59:59',"<<--custom time"] ][projected]

# Run function to create pivot tables
periodData            = timeSliceData(tPeriod,tPeriod2,dataSlice1,dataSliceOpn)

## Price and VWAP Pivot Tables 
P_,    P_Proj         = periodData[0], periodData[1]
PVWAP ,PVWAPProj      = periodData[2], periodData[3]


## Inputs for KDB.AI using HNSW and L2 similarity
## vecs object is the vector embeddings.
Date_                 = P_.index              ## Dates
vecs                  = P_.values.tolist()    ## vectors
dims                  = len(vecs[0])          ## dims for schema 

In [ ]:
## Combine the two dataframes
embeddingsDF = pd.DataFrame({"date":    pd.to_datetime(Date_), 
                             "sym":     filtersData["Symbol"].values.tolist(), 
                             "DOW":     filtersData["DOW"].values.astype('int32').tolist(),  
                             "RF":      filtersData["RF"].values.astype('int32').tolist(), 
                             "WOM":     filtersData["WOM"].values.astype('int32').tolist(),
                             "OpExRF":  filtersData["OpExRF"].values.astype('int32').tolist(), 
                             "vectors": vecs})

# embeddingsDF.info() ### shows dtypes as int64

In [ ]:
## check embeddings
show_df(embeddingsDF)

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have seven colums: date, sym, DOW, RF, WOM, OpExRF, and vectors. 

In [ ]:
## Create schema for KDB.AI cloud instance 
## Notice the similiarities to DataFrame construction  and "*DataFrame*".info()
stockSchema = {'columns': [{'name': 'date',      'pytype': 'datetime64[ns]', },
                            {'name': 'sym',      'pytype':  'str',  },
                            {'name': 'DOW',      'pytype': 'int32', },
                            {'name': 'RF',       'pytype': 'int32', },
                            {'name': 'WOM',      'pytype': 'int32', },
                            {'name': 'OpExRF',   'pytype': 'int32', },
                            {'name': 'vectors',  'vectorIndex': {'dims': dims, 'metric': 'L2', 'type': 'hnsw'}, },]}

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [ ]:
# Remove Drop table from cloud instance. This does not need to done if you are going to reuse the table. 
# In this notebook, we follow the practice of dropping table after use. 
TableCheckDrop("Stock")

In [ ]:
table = session.create_table("Stock", stockSchema)

### Add Embedded Data to KDB.AI Session Table 


In [ ]:
## Check the size of data. 
embeddingsDF.memory_usage(deep=True).sum()/(1024**2)  # Convert bytes to MB

Since embeddings are small <10MB, we will insert without chunking.  See others KDB.AI examples if greater than 10MB

In [ ]:
# Verify the data has been inserted using the method table.query() which shows us that data has been added without errors
# Sanity check raw Query
table.insert(embeddingsDF)
show_df(table.query())

### Search KDB.AI Based on this Example Pattern

We can extract the vectors from the example window selected above into a `query_vector` and search the vector database to get the five nearest other windows to this pattern.


In [ ]:
## With vectors mapped, uploaded and indexed, you can search for dates. We primarly use HNSW index values to select days 
## This notebook can be is easily modified to explore unseen data. 
## Since we are filtering after uploading and indexing, the index value shouldn't change.
## If we instead filtered prior to uploading and indexing, the index value will change. 
## Sanity Check 
print(findIndex('2021-12-15'))
print(findDate(650))

In [ ]:
# This target is search vector. This can be replace with data that isn't in the embeddings vector uploaded.
# If new unseen vectors are used, some of the refDates calculation will need to be modified.  
findVector   = [vecs[650]]
# print(findVector)

In [ ]:
##  "findVector" from above , "n=9" the number of nearest neighbors, we have requested nine. 
matchedVectors = table.search(findVector, n=9)

## We can use filter or multiple filters after indexing. (N.B. We can also pre-filter data prior to uploading and indexing). 
# matchedVectors = table.search(findVector, n=9, filter=[("<>", "WOM", 3)])
# matchedVectors = table.search(findVector, n=9, filter=[("=", "OpExRF", 0)])
display(matchedVectors[0].head(3))

In [ ]:
dates      = pd.DataFrame(matchedVectors[0]["date"]);
refDates   = dates["date"].dt.date.astype('str'); 
# matchedVectors

In [ ]:
### Plot the data. This is the single axis version of the plot.

title = "Indexed Price Data - Single Axis for %s"  % (refDates[0])
P_.loc[refDates[0]].plot(color='black', title= title,label="Close Data" )
P_.loc[refDates[1:]].mean().plot(label="NN Mean Close Prices")
PVWAP.loc[refDates[0]].plot(color='green',label="VWAP").legend(loc='best');

In [ ]:
### This is the dual axes version of the plot. There sometimes a better visual display of the data.

p1date      = P_.loc[refDates[0]].index.astype(str) 
p1data      = P_.loc[refDates[0]].values
p1Vdata     = PVWAP.loc[refDates[0]].values
p2data      = P_.loc[refDates[1:]].mean().values

title       = "Indexed Price Data - Dual Axes"
plot2Axes(p1date,p1data,p2data,p1Vdata,refDates[0],title,"window");

In [ ]:
###  Plot the projected data based on NN's. This is the single axis version of the plot. We projecting using the mean data from the nine NN
## 

title = "Projected Indexed Price Data - Single Axis for %s" % (refDates[0])
P_Proj.loc[refDates[0]].plot(color='brown', title=title,label="Close Data")
P_Proj.loc[refDates[1:]].mean().plot(label="NN Mean Close Prices")
PVWAPProj.loc[refDates[0]].plot(color='green',label="VWAP").legend(loc='best');

In [ ]:
### Plot the projected data based on NN's. This is the dual axes version of the plot. There sometimes a better visual display of the data.

Eval2date       = P_Proj.loc[refDates[0]].index.astype(str) 
Eval2data       = P_Proj.loc[refDates[0]].values
Eval2dataM      = P_Proj.loc[refDates[1:]].mean().values
Eval2VWAPdata   = PVWAPProj.loc[refDates[0]].values

title           = "Projected Indexed Price Data  - Dual Axes"
plot2Axes(Eval2date,Eval2data,Eval2dataM,Eval2VWAPdata,refDates[0],title,"projection" ,c = 'r-');

In [ ]:
# Plot showing the nine NN plots for the mapped (P1) time period 
P_.index     = P_.index.astype('str');
Eval2NN      = P_.T[refDates[1:]];
title_       = "NN Close Prices for %s" % (refDates[0]);

Eval2NN.plot(title =title_ );

In [ ]:
# Plot is showing the nine NN plots for the projected time period.  

P_Proj.index = P_Proj.index.astype('str')
Eval2NN      = P_Proj.T[refDates[1:]]
title_2      = "NN Projected Close Prices for %s" % (refDates[0])
# Eval2NN.legend(loc='lower right')
Eval2NN.plot(title =title_2 ).legend(loc='lower right');

### Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [ ]:
# table.drop()